In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from matplotlib.backends.backend_pdf import PdfPages
pd.set_option('use_inf_as_na', True)
pd.set_option('mode.chained_assignment', None)
sns.set(context = 'paper', style='whitegrid', font_scale=1.8, rc = {'axes.spines.right':False, 'axes.spines.top': False, 'figure.figsize':(12.7,8.6)}, palette='Set1')


# Before you begin

1. Update save_path with your local directory where you wish to save the tables and figures you generate
2. Update path_template with your local directory where you have saved your microsimulation results
3. Update the mapping_dict_data['model_7_results'] dictionary with the directory filepaths in your local directory

In [ ]:
mapping_dict_data = {'cause': {'acute_myocardial_infarction':'ischemic_heart_disease','post_myocardial_infarction':'ischemic_heart_disease','acute_ischemic_stroke':'ischemic_stroke','post_ischemic_stroke':'ischemic_stroke', 'ischemic_stroke':'ischemic_stroke', 'chronic_kidney_disease':'chronic_kidney_disease', 'ischemic_heart_disease':'ischemic_heart_disease', 'diabetes_mellitus':'diabetes_mellitus', 'other_causes':'other_causes', 'albuminuria':'chronic_kidney_disease', 'stage_iii_chronic_kidney_disease':'chronic_kidney_disease','stage_iv_chronic_kidney_disease':'chronic_kidney_disease','stage_v_chronic_kidney_disease':'chronic_kidney_disease','moderate_diabetes_mellitus':'diabetes_mellitus','severe_diabetes_mellitus':'diabetes_mellitus', 'chronic_kidney_disease':'chronic_kidney_disease', 'CVD':'CVD'},
                    'cause_client': {'ischemic_heart_disease':'Ischemic heart disease', 'ischemic_stroke':'Ischemic stroke', 'chronic_kidney_disease':'Chronic kidney disease', 'diabetes_mellitus':'Diabetes mellitus', 'other_causes':'Other causes', 'CVD':'CVD'},
                    'sequela':{378:'acute_myocardial_infarction', 379:'acute_myocardial_infarction',380:'post_myocardial_infarction', 381:'post_myocardial_infarction', 382:'post_myocardial_infarction', 383:'post_myocardial_infarction', 384:'post_myocardial_infarction', 385:'post_myocardial_infarction', 953:'post_myocardial_infarction', 1040:'post_myocardial_infarction', 5726:'post_myocardial_infarction', 386: 'acute_ischemic_stroke', 387: 'acute_ischemic_stroke', 388: 'acute_ischemic_stroke', 389: 'acute_ischemic_stroke', 390: 'acute_ischemic_stroke', 391:'post_ischemic_stroke', 392: 'post_ischemic_stroke', 393:'post_ischemic_stroke', 394:'post_ischemic_stroke', 395:'post_ischemic_stroke', 946:'post_ischemic_stroke'},
                    'location': {'china':'China', 'brazil':'Brazil', 'france':'France', 'italy':'Italy', 'russian_federation':'Russia', 'spain':'Spain', 'All locations':'All locations'},
                    'sex':{'female':'female','male':'male', 'both_sexes_combined':'both sexes'},
                    'scenario': {'baseline':'Business as Usual', 'guideline':'Intervention 1 (multiple pills)', 'guideline_and_new_treatment':'Intervention 2 (FDC)'},
                    'measure':{'death':'deaths','ylds':'ylds','ylls':'ylls'},
                    'measure_client':{'deaths':'Deaths', 'incidence':'Incidence'},
                    'model_7_results':{'brazil':'brazil/your_directory', 'china':'china/your_directory','france':'france/your_directory', 'italy':'italy/your_directory', 'russian_federation':'russian_federation/your_directory', 'spain':'spain/your_directory'}}

locations = ['brazil', 'china', 'france', 'italy','russian_federation', 'spain']

measures = ['deaths', 'person_time','population','ylds','ylls', 'prevalence', 'incidence']
save_path_j = 'filepath'

p = save_path_j
p.mkdir(parents=True, exist_ok=True)


# Defining functions for dataframes
* Structed to retrieve specific dataframes from model outputs

In [ ]:
path_template = Path('filepath')

def get_model_outputs_sequela(locations, measure):
    outcomes = pd.DataFrame([])
    try:   
        for location in locations:
            country_loc = mapping_dict_data['model_7_results'][location]
            outcome = pd.read_hdf(str(path_template / f'{country_loc}/count_data/{measure}.hdf'))
            outcome['location'] = f'{location}'
            if measure != 'transition_count':
                outcome['measure'] = f'{measure}'
            outcomes = outcomes.append(outcome)
        return outcomes
            
    except:
        print(f'{measure} is not a measure in the outputs')

def get_model_outputs_cause(locations, measure):
    outcomes = pd.DataFrame([])
    try:   
    
        for location in locations:
            country_loc = mapping_dict_data['model_7_results'][location]
            outcome = pd.read_hdf(str(path_template / f'{country_loc}/count_data/{measure}.hdf'))
            outcome['location'] = f'{location}'
        
            if measure != 'transition_count':
                outcome['measure'] = f'{measure}'
            outcomes = outcomes.append(outcome)
            outcomes['cause'] = outcomes.cause.map(mapping_dict_data['cause'])
            outcomes = outcomes.dropna()

        return outcomes
            
    except:
            print(f'{measure} is not a measure in the outputs')



In [ ]:
def get_pops(locations):
    outcomes = pd.DataFrame([])
    for country in locations:
        country_loc = mapping_dict_data['model_7_results'][country]
        outcome = pd.read_hdf(str(path_template / f'{country_loc}/count_data/population.hdf'))
        outcome['location'] = f'{country}'
        outcomes = outcomes.append(outcome)
    return outcomes

def get_person_time(locations):
    outcomes = pd.DataFrame([])
    for country in locations:
        country_loc = mapping_dict_data['model_7_results'][country]
        outcome = pd.read_hdf(str(path_template / f'{country_loc}/count_data/person_time.hdf'))
        outcome['location'] = f'{country}'
        outcomes = outcomes.append(outcome)
    return outcomes

def get_person_time_misc(locations):
    outcomes = pd.DataFrame([])
    for country in locations:
        country_loc = mapping_dict_data['model_7_results'][country]
        outcome = pd.read_hdf(str(path_template / f'{country_loc}/count_data/miscellaneous_person_time.hdf'))
        outcome['location'] = f'{country}'
        outcomes = outcomes.append(outcome)
    return outcomes

def get_person_time_rfs(locations):
    outcomes = pd.DataFrame([])
    for country in locations:
        country_loc = mapping_dict_data['model_7_results'][country]
        outcome = pd.read_hdf(str(path_template / f'{country_loc}/count_data/person_time.hdf'))
        outcome['location'] = f'{country}'
        outcomes = outcomes.append(outcome)
    return outcomes
def get_transition(locations):
    outcomes = pd.DataFrame([])
    for country in locations:
        country_loc = mapping_dict_data['model_7_results'][country]
        df = pd.read_hdf(str(path_template / f'{country_loc}/count_data/transition_count.hdf'))
        df['location'] = f'{country}'
        outcomes = outcomes.append(df)
    return outcomes

def get_transition_susceptible_to_acute(locations):
    transitions = get_model_outputs_sequela(locations, 'transition_count')
    transitions = transitions[transitions.measure.str.startswith('susceptible')]
    transitions['susceptible'] = transitions.measure.str.extract('((?<=to_).*?(?=_to))',expand = False)
    transitions['cause'] = transitions.measure.str.extract('((?<=to_).*?(?=_event_count))',expand = False)
    transitions['cause'] = transitions.cause.str.extract('((?<=_to_).*)', expand = False)
    transitions = transitions.drop('measure', axis = 1)
    return transitions

def get_transition_post_to_acute(locations):    
    transitions = get_model_outputs_sequela(locations, 'transition_count')
    transitions = transitions[transitions.measure.str.startswith('post')]
    transitions['susceptible'] = transitions.measure.str.extract('(.*?(?=_to))',expand = False)
    transitions['cause'] = transitions.measure.str.extract('((?<=to_).*?(?=_event_count))',expand = False)
    transitions = transitions.drop('measure', axis = 1)
    return transitions



# Assigning dataframes to be used in various calculations and figures

In [ ]:
person_time_rfs = get_person_time_rfs(locations)
person_time = get_person_time(locations)
susceptible_to_acute = get_transition_susceptible_to_acute(locations)
post_to_acute = get_transition_post_to_acute(locations)
ylds = get_model_outputs_cause(locations, 'ylds')
ylds['measure'] = 'ylds'
ylls = get_model_outputs_cause(locations, 'ylls')
ylls['measure'] = 'ylls'
deaths = get_model_outputs_cause(locations, 'deaths')
deaths['measure'] = 'deaths'
state = get_model_outputs_cause(locations, 'state_person_time')
state['measure'] = 'prevalence'
person_time_misc = get_person_time_misc(locations)
pops = get_pops(locations)

In [ ]:
def collapse_frame(df, column):
    nonidcols = ['value']
    nonidcols.extend(column)
    return(df.groupby([x for x in df.columns if x not in nonidcols])['value'].sum().reset_index())

In [ ]:
def get_rf_data(person_time_misc, person_time_rfs, risk_factor, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, year_strat=True, diabetes_strat=True,htn_strat=True, loc_strat=True, high_ldl_strat=True):
        rf = person_time_misc[(person_time_misc.measure == f'{risk_factor}_person_time')]
        rf['measure'] = f'{risk_factor}'
        rf['subpopulation'] = 'total_population'
        person_time = person_time_rfs
        person_time['subpopulation'] = 'total_population'
        person_time = person_time.drop(columns=['measure'])
        rf = rf[(rf.age_group != '30_to_34') & (rf.age_group != '35_to_39') & (rf.age_group != '85_to_89') & (rf.age_group != '90_to_94') & (rf.age_group != '95_plus')]
        person_time = person_time[(person_time.age_group != '30_to_34') & (person_time.age_group != '35_to_39') & (person_time.age_group != '85_to_89') & (person_time.age_group != '90_to_94') & (person_time.age_group != '95_plus')]

        if not age_strat_all_ages:
            rf = collapse_frame(rf, ['age_group'])
            rf['age_group'] = 'All_ages'
            person_time = collapse_frame(person_time, ['age_group'])
            person_time['age_group'] = 'All_ages'
        if not age_strat_75plus:
            rf_75 = rf[(rf.age_group == '75_to_79') | (rf.age_group == '80_to_84')]
            rf = collapse_frame(rf_75, ['age_group'])
            rf['age_group'] = '75_plus'
            pt_75 = person_time[(person_time.age_group == '75_to_79') | (person_time.age_group == '80_to_84')]
            person_time = collapse_frame(pt_75, ['age_group'])
            person_time['age_group'] = '75_plus'
        if not age_strat_40_74:
            rf_40_74 = rf[rf.age_group.isin(['40_to_44', '45_to_49', '50_to_54', '55_to_59', '60_to_64', '65_to_69', '70_to_74'])]
            rf = collapse_frame(rf_40_74, ['age_group'])
            rf['age_group'] = '40_to_74'
            pt_40_74 = person_time[person_time.age_group.isin(['40_to_44', '45_to_49', '50_to_54', '55_to_59', '60_to_64', '65_to_69', '70_to_74'])]
            person_time = collapse_frame(pt_40_74, ['age_group'])
            person_time['age_group'] = '40_to_74'
        if not sex_strat:
            rf = collapse_frame(rf, ['sex'])
            rf['sex'] = 'both_sexes_combined'
            person_time = collapse_frame(person_time, ['sex'])
            person_time['sex'] = 'both_sexes_combined'
        if not year_strat:
            rf = collapse_frame(rf, ['year'])
            rf['year'] = 'All years'
            person_time = collapse_frame(person_time, ['year'])
            person_time['year'] = 'All years'
        if not risk_strat:
            rf = collapse_frame(rf,['acs','fpg', 'sbp', 'ldl'])
            person_time = collapse_frame(person_time, ['acs','fpg', 'sbp', 'ldl'])
        if not diabetes_strat:
            rf_dm = rf[(rf.fpg=='high')]
            rf = collapse_frame(rf_dm, ['acs','fpg', 'sbp', 'ldl'])
            rf['subpopulation'] = 'diabetic'
            person_time_dm = person_time[(person_time.fpg=='high')]
            person_time = collapse_frame(person_time_dm, ['acs','fpg', 'sbp', 'ldl'])
            person_time['subpopulation'] = 'diabetic'
        if not htn_strat:
            rf_htn = rf[(rf.sbp=='high')]
            rf = collapse_frame(rf_htn, ['acs','fpg', 'sbp', 'ldl'])
            rf['subpopulation'] = 'hypertensive'
            person_time_htn = person_time[(person_time.sbp=='high')]
            person_time = collapse_frame(person_time_htn, ['acs','fpg', 'sbp', 'ldl'])
            person_time['subpopulation'] = 'hypertensive'
        if not loc_strat:
            rf = collapse_frame(rf, ['location'])
            rf['location'] = 'All locations'
            person_time = collapse_frame(person_time, ['location'])
            person_time['location'] = 'All locations'
        if not high_ldl_strat:
            rf_ldl = rf[(rf.ldl == 'high')]
            rf = collapse_frame(rf_ldl, ['acs','fpg', 'sbp', 'ldl'])
            rf['subpopulation'] = 'high_ldl'
            person_time_ldl = person_time[(person_time.ldl=='high')]
            person_time = collapse_frame(person_time_ldl, ['acs','fpg', 'sbp', 'ldl'])
            person_time['subpopulation'] = 'high_ldl'
            
        rf = rf.set_index([x for x in person_time.columns if x != 'value'] + ['measure']).value
        person_time = person_time.set_index([x for x in person_time.columns if x != 'value']).value
        rf_mean_val = (rf / person_time).reset_index()
        rf_mean_val = rf_mean_val.dropna()
        rf_mean_val['location'] = rf_mean_val.location.map(mapping_dict_data['location'])
        rf_mean_val['scenario'] = rf_mean_val.scenario.map(mapping_dict_data['scenario'])
        rf_mean_val = rf_mean_val.rename(columns={'scenario':'Scenario'})
        return rf_mean_val

# To get the LDL-c table
* Stratified by risk subpopulation (High LDL, Hypertensive, Diabetic, Entire Population)
* High LDL: 'all ages', all age groups, 40_74, 75+, male, female, both
* htn: 'all ages', all age groups, 40_74, 75+, male, female, both
* diabetes: 'all ages', all age groups, 40_74, 75+, male, female, both
* entire population: 'all ages', all age groups, 40_74, 75+, male, female, both



In [ ]:
# #htn strat
ldlc_rf_htn_allages = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=True, loc_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, diabetes_strat=True, htn_strat=False, high_ldl_strat=True)
ldlc_rf_htn_75plus = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=True, loc_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, diabetes_strat=True, htn_strat=False, high_ldl_strat=True)
ldlc_rf_htn_allages_both = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=True, loc_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, diabetes_strat=True, htn_strat=False, high_ldl_strat=True)
ldlc_rf_htn_75plus_both = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=True, loc_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, diabetes_strat=True, htn_strat=False, high_ldl_strat=True)
ldlc_rf_htn_70to74 = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=True, loc_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, diabetes_strat=True, htn_strat=False, high_ldl_strat=True)
ldlc_rf_htn_70to74_both = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=True, loc_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, diabetes_strat=True, htn_strat=False, high_ldl_strat=True)
ldlc_rf_htn = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=True, loc_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, diabetes_strat=True, htn_strat=False, high_ldl_strat=True)
ldlc_rf_htn_both = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=True, loc_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, diabetes_strat=True, htn_strat=False, high_ldl_strat=True)

# #diabetes strat
ldlc_rf_dm_allages = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=True, loc_strat=True, age_strat_all_ages=False,age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, diabetes_strat=False, htn_strat=True, high_ldl_strat=True)
ldlc_rf_dm_75plus = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=True, loc_strat=True, age_strat_all_ages=True,age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, diabetes_strat=False, htn_strat=True, high_ldl_strat=True)
ldlc_rf_dm_allages_both = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=True, loc_strat=True, age_strat_all_ages=False,age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, diabetes_strat=False, htn_strat=True, high_ldl_strat=True)
ldlc_rf_dm_75plus_both = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=True, loc_strat=True, age_strat_all_ages=True,age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, diabetes_strat=False, htn_strat=True, high_ldl_strat=True)
ldlc_rf_dm_70to74 = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=True, loc_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, diabetes_strat=False, htn_strat=True, high_ldl_strat=True)
ldlc_rf_dm_70to74_both = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=True, loc_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, diabetes_strat=False, htn_strat=True, high_ldl_strat=True)
ldlc_rf_dm = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=True, loc_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, diabetes_strat=False, htn_strat=True, high_ldl_strat=True)
ldlc_rf_dm_both = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=True, loc_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, diabetes_strat=False, htn_strat=True, high_ldl_strat=True)

#High-LDL strat
ldlc_rf_high_ldl_allages = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=True, loc_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, diabetes_strat=True,htn_strat=True, high_ldl_strat=False)
ldlc_rf_high_ldl_allages_both = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=True, loc_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, diabetes_strat=True,htn_strat=True, high_ldl_strat=False)
ldlc_rf_high_ldl_75plus = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=True, loc_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, diabetes_strat=True,htn_strat=True, high_ldl_strat=False)
ldlc_rf_high_ldl_75plus_both = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=True, loc_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, diabetes_strat=True,htn_strat=True, high_ldl_strat=False)
ldlc_rf_high_ldl_70to74 = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=True, loc_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, diabetes_strat=True, htn_strat=True, high_ldl_strat=False)
ldlc_rf_high_ldl_70to74_both = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=True, loc_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, diabetes_strat=True, htn_strat=True, high_ldl_strat=False)
ldlc_rf_high_ldl = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=True, loc_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, diabetes_strat=True, htn_strat=True, high_ldl_strat=False)
ldlc_rf_high_ldl_both = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=True, loc_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, diabetes_strat=True, htn_strat=True, high_ldl_strat=False)

#no risk strat
ldlc_rf_total_all_ages = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=False, loc_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, diabetes_strat=True, htn_strat=True, high_ldl_strat=True)
ldlc_rf_total_both_all_ages = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=False, loc_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, diabetes_strat=True, htn_strat=True, high_ldl_strat=True)
ldlc_rf_total_75plus = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=False, loc_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, diabetes_strat=True, htn_strat=True, high_ldl_strat=True)
ldlc_rf_total_75plus_both = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=False, loc_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, diabetes_strat=True, htn_strat=True, high_ldl_strat=True)
ldlc_rf_total_70to74 = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=False, loc_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, diabetes_strat=True, htn_strat=True, high_ldl_strat=True)
ldlc_rf_total_70to74_both = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=False, loc_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, diabetes_strat=True, htn_strat=True, high_ldl_strat=True)
ldlc_rf_total = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=False, loc_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, diabetes_strat=True, htn_strat=True, high_ldl_strat=True)
ldlc_rf_total_both = get_rf_data(person_time_misc, person_time_rfs, 'ldlc', risk_strat=False, loc_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, diabetes_strat=True, htn_strat=True, high_ldl_strat=True)

all_ldl = pd.concat([ldlc_rf_htn_allages, ldlc_rf_htn_75plus, ldlc_rf_htn_allages_both, ldlc_rf_htn_75plus_both, ldlc_rf_htn_allages, ldlc_rf_htn_75plus, ldlc_rf_htn_allages_both, ldlc_rf_htn_75plus_both, ldlc_rf_htn_70to74, ldlc_rf_htn_70to74_both, ldlc_rf_htn, ldlc_rf_htn_both, ldlc_rf_dm_allages, ldlc_rf_dm_75plus, ldlc_rf_dm_allages_both, ldlc_rf_dm_75plus_both, ldlc_rf_dm_70to74, ldlc_rf_dm_70to74_both, ldlc_rf_dm, ldlc_rf_dm_both, ldlc_rf_high_ldl_allages, ldlc_rf_high_ldl_allages_both, ldlc_rf_high_ldl_75plus, ldlc_rf_high_ldl_75plus_both, ldlc_rf_high_ldl_70to74, ldlc_rf_high_ldl_70to74_both, ldlc_rf_high_ldl, ldlc_rf_high_ldl_both, ldlc_rf_total_all_ages, ldlc_rf_total_both_all_ages, ldlc_rf_total_75plus, ldlc_rf_total_75plus_both, ldlc_rf_total_70to74, ldlc_rf_total_70to74_both, ldlc_rf_total, ldlc_rf_total_both])


In [ ]:
def get_difference_from_baseline(all_ldl):
    shared_columns = ['age_group','sex','year', 'location', 'measure','Scenario', 'subpopulation']
    df = all_ldl

    baseline = df[df.Scenario == 'Business as Usual']
    baseline = baseline.rename(columns = {'value':'baseline'})
    df = pd.merge(df, baseline, left_on=['age_group', 'sex', 'year', 'input_draw', 'location', 'measure', 'subpopulation'], right_on=['age_group', 'sex', 'year', 'input_draw', 'location', 'measure', 'subpopulation'], how='left')
    df['difference_from_baseline_count'] = df.apply(lambda row: (row['baseline'] - row['value']),axis=1)
    df = df.rename(columns={'Scenario_x':'Scenario'})
    df = df.drop(columns=['Scenario_y', 'baseline'])
    return df

df = get_difference_from_baseline(all_ldl)

def quantile_(q):
    """Factory function for pandas gb aggregate.
    """
    def q_factory(df):
        return df.quantile(q)
    q_factory.__name__ = f"Q{q:0.3f}"
    return q_factory

def report_aggregates(df):
    gb = df.groupby(["age_group", "sex", "year", "location", "measure", "Scenario", "subpopulation"], as_index=False)
    metrics = ["mean", quantile_(0.025), quantile_(0.975)]
    df = gb.agg({"value": metrics, "difference_from_baseline_count": metrics})
    df.columns = list(map(lambda c: "_".join(filter(None, c)), df.columns))
    return df

aggregates = report_aggregates(df)

In [ ]:
def ldl_table_formatting(aggregates):
    df = aggregates
    df['subpopulation'] = df.subpopulation.str.replace('high_ldl','LDL-c > 4.9')
    df['subpopulation'] = df.subpopulation.str.replace('hypertensive','Hypertension (SBP > 140 mmHg)')
    df['subpopulation'] = df.subpopulation.str.replace('diabetic','Diabetes (FPG > 7.0 mmol/l)')
    df['subpopulation'] = df.subpopulation.str.replace('total_population','Total population')
    df['sex'] = df.sex.map(mapping_dict_data['sex'])
    df['sex'] = df.sex.str.capitalize()
    df = df.rename(columns={'measure':'Measure'})
    df = df[(df.age_group != '75_to_79') & (df.age_group != '80_to_84')]
    df['age_group'] = df.age_group.str.replace('_', ' ')
    df['age_group'] = df.age_group.str.replace(' plus', '+')
    df['age_group'] = df.age_group.str.replace('All ages', '40+')

    df = df.rename(columns={'age_group':'Age group', 'sex':'Sex', 'location':'Location name', 
                            'subpopulation':'Subpopulation', 'year':'Year', 
                            'value_mean':'Mean LDL-c estimated value', 
                            'value_Q0.025':'Lower LDL-c uncertainty interval', 
                            'value_Q0.975':'Upper LDL-c uncertainty interval', 
                            'difference_from_baseline_count_mean':'Difference from baseline- LDL-c mean estimated value',
                            'difference_from_baseline_count_Q0.025':'Difference from baseline- lower LDL-c uncertainty interval', 
                            'difference_from_baseline_count_Q0.975':'Difference from baseline- upper LDL-c uncertainty interval'})
    df = df[['Location name', 'Year', 'Subpopulation', 'Age group', 'Sex', 'Scenario', 'Mean LDL-c estimated value', 'Lower LDL-c uncertainty interval', 'Upper LDL-c uncertainty interval', 'Difference from baseline- LDL-c mean estimated value', 'Difference from baseline- lower LDL-c uncertainty interval', 'Difference from baseline- upper LDL-c uncertainty interval']]
    df['Sex'] = df.Sex.str.replace('Both sexes','Both')
    return df

ldl_table = ldl_table_formatting(aggregates)

# Output LDL-c supplemental data table

In [ ]:
ldl_table.to_csv(str(save_path_j)+'/ldl_c_supplemental_data_output_table.csv', index=False)


# Figures for LDL-c table
##### If desired, this function can be customized to add subpopulations of:
* 'LDL-c > 4.9'
* 'Hypertension (SBP > 140 mmHg)'
* 'Diabetes (FPG > 7.0 mmol/l)'

* To do this, add the desired subpopulations to the 'subpopulations' list within the function below

In [ ]:
def plot_ldl_c_mean_location_specific(ldl_table):
    ldlc_rf = ldl_table

    locations = ['Brazil', 'China', 'France', 'Italy','Russia', 'Spain']
    subpopulations = ['Entire population']
    ldlc_rf['Subpopulation'] = ldlc_rf['Subpopulation'].str.replace('Total population', 'Entire population')
    scenario_list = ['Business as Usual','Intervention 1 (multiple pills)', 'Intervention 2 (FDC)']
    palette_list = ['#B04F6C','#6CB04F','#4F6CB0']
    palette_dict = {x:v for x,v in zip(scenario_list,palette_list)}
    
    with PdfPages(str(save_path_j)+'/mean_ldl_c_over_time_model_outputs_location.pdf') as pdf:
        for location in locations: 
            for subpop in subpopulations:

                df_plot = ldlc_rf[(ldlc_rf['Location name'] == location) & (ldlc_rf['Subpopulation'] == subpop)]

                g = sns.lineplot(x='Year', y='Mean LDL-c estimated value', hue = 'Scenario', palette= palette_dict, legend=False,
                                 linewidth=4, data=df_plot)

                g.set_title(f'Mean LDL-c over time for \n {subpop}, {location}')
                g.set_xlabel('Year')
                g.set_ylabel(f'LDL-c (mmol/L)')

                pdf.savefig(orientation = 'landscape', bbox_inches='tight')

            plt.show()
            plt.clf() 

plot_ldl_c_mean_location_specific(ldl_table)

# To get the full data output table
* Incidence is calculated separately from all other measures


In [ ]:
def get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include = None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=True, year_strat=True):
    transitions = pd.concat([susceptible_to_acute, post_to_acute])
    transitions['subpopulation'] = 'total_population'
    state = get_model_outputs_sequela(locations, 'state_person_time')
    state['subpopulation'] = 'total_population'
    transitions = transitions[(transitions.age_group != '30_to_34') & (transitions.age_group != '35_to_39')]
    state = state[(state.age_group != '30_to_34') & (state.age_group != '35_to_39')]
    state = state[(state.cause.str.contains('susceptible')) | (state.cause.str.contains('post'))]
    state['susceptible'] = state.cause.str.replace('susceptible_to_', '')
    if causes_to_include != None:
        transitions = transitions[transitions.susceptible.isin(causes_to_include)]
        state = state[state.susceptible.isin(causes_to_include)]
    state = state.drop(['cause', 'measure'], axis = 1)
    if not risk_strat:
        state = collapse_frame(state, ['acs','fpg', 'sbp', 'ldl'])
        transitions = collapse_frame(transitions, ['acs','fpg', 'sbp', 'ldl'])
    if not age_strat_all_ages:
        transitions = collapse_frame(transitions, ['age_group'])
        transitions['age_group'] = '40_plus'
        state = collapse_frame(state, ['age_group'])
        state['age_group'] = '40_plus'
    if not age_strat_75plus:
        transitions_75 = transitions[transitions.age_group.isin(['75_to_79', '80_to_84', '85_to_89', '90_to_95', '95_plus'])]
        transitions = collapse_frame(transitions_75, ['age_group'])
        transitions['age_group'] = '75_plus'
        state_75 = state[state.age_group.isin(['75_to_79', '80_to_84', '85_to_89', '90_to_95', '95_plus'])]
        state = collapse_frame(state_75, ['age_group'])
        state['age_group'] = '75_plus'
    if not age_strat_40_74:
        transitions_40_74 = transitions[transitions.age_group.isin(['40_to_44', '45_to_49', '50_to_54', '55_to_59', '60_to_64', '65_to_69', '70_to_74'])]
        transitions = collapse_frame(transitions_40_74, ['age_group'])
        transitions['age_group'] = '40_to_74'
        state_40_74 = state[state.age_group.isin(['40_to_44', '45_to_49', '50_to_54', '55_to_59', '60_to_64', '65_to_69', '70_to_74'])]
        state = collapse_frame(state_40_74, ['age_group'])
        state['age_group'] = '40_to_74'
    if not sex_strat:
        transitions = collapse_frame(transitions, ['sex'])
        transitions['sex'] = 'both_sexes_combined'
        state = collapse_frame(state, ['sex'])
        state['sex'] = 'both_sexes_combined'
    if not cvd_strat:
        transitions = collapse_frame(transitions, ['cause'])
        transitions['susceptible'] = 'CVD'
        state = collapse_frame(state, ['susceptible'])
        state['susceptible'] = 'CVD'
    if not loc_strat:
        transitions = collapse_frame(transitions, ['location'])
        transitions['location'] = 'All locations'
        state = collapse_frame(state, ['location'])
        state['location'] = 'All locations'
    if not diabetes_strat:
        transitions_dm = transitions[(transitions.fpg=='high')]
        transitions = collapse_frame(transitions_dm, ['acs','fpg', 'sbp', 'ldl'])
        transitions['subpopulation'] = 'diabetic'
        state_dm = state[(state.fpg=='high')]
        state = collapse_frame(state_dm, ['acs','fpg', 'sbp', 'ldl'])
        state['subpopulation'] = 'diabetic'
    if not htn_strat:
        transitions_htn = transitions[(transitions.sbp=='high')]
        transitions = collapse_frame(transitions, ['acs','fpg', 'sbp', 'ldl'])
        transitions['subpopulation'] = 'hypertensive'
        state_htn = state[(state.sbp=='high')]
        state = collapse_frame(state_htn, ['acs','fpg', 'sbp', 'ldl'])
        state['subpopulation'] = 'hypertensive' 
    if not high_ldl_strat:
        transitions_ldl = transitions[(transitions.ldl=='high')]
        transitions = collapse_frame(transitions_ldl, ['acs','fpg', 'sbp', 'ldl'])
        transitions['subpopulation'] = 'high_ldl'
        state_ldl = state[(state.ldl=='high')]
        state = collapse_frame(state_ldl, ['acs','fpg', 'sbp', 'ldl'])
        state['subpopulation'] = 'high_ldl'
    if not year_strat:
        transitions = collapse_frame(transitions, ['year'])
        transitions['year'] = 'All years'
        state = collapse_frame(state, ['year'])
        state['year'] = 'All years'
    if not final_cause_strat:
        transitions = collapse_frame(transitions, ['cause'])
        transitions = transitions.set_index([x for x in state.columns if x != 'value']).value
    else:
        transitions = transitions.set_index([x for x in state.columns if x != 'value'] + ['cause']).value
    state = state.set_index([x for x in state.columns if x != 'value']).value
    incidence = (transitions/state * 100_000).reset_index()
    incidence['measure'] = 'incidence'
    if 'cause' not in incidence.columns:
        incidence.rename(columns = {'susceptible':'cause'}, inplace = True)
    incidence['cause'] = incidence.cause.map(mapping_dict_data['cause'])

    return incidence

# To get the full data output table
* Stratified by risk subpopulation (High LDL, Hypertensive, Diabetic, Entire Population)
* htn: 'all ages', all age groups, 40_74, 75+, male, female, both
* diabetes: 'all ages', all age groups, 40_74, 75+, male, female, both
* entire population: 'all ages', all age groups, 40_74, 75+, male, female, both



In [ ]:
# htn 
incidence_htn_all_ages = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = False, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_htn_all_ages_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None,  age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, loc_strat=True, htn_strat = False, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_htn_40to74 = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, loc_strat=True, htn_strat = False, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_htn_40to74_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None,  age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, loc_strat=True, htn_strat = False, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_htn_75plus = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = False, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_htn_75plus_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, loc_strat=True, htn_strat = False, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_htn = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = False, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_htn_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, loc_strat=True, htn_strat = False, diabetes_strat = True, high_ldl_strat=True, year_strat=True)


# #diabetes
incidence_db_all_ages = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = False, high_ldl_strat=True, year_strat=True)
incidence_db_all_ages_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, loc_strat=True, htn_strat = True, diabetes_strat = False, high_ldl_strat=True, year_strat=True)
incidence_db_40to74 = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = False, high_ldl_strat=True, year_strat=True)
incidence_db_40to74_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, loc_strat=True, htn_strat = True, diabetes_strat = False, high_ldl_strat=True, year_strat=True)
incidence_db_75plus = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = False, high_ldl_strat=True, year_strat=True)
incidence_db_75plus_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, loc_strat=True, htn_strat = True, diabetes_strat = False, high_ldl_strat=True, year_strat=True)
incidence_db = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = False, high_ldl_strat=True, year_strat=True)
incidence_db_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, loc_strat=True, htn_strat = True, diabetes_strat = False, high_ldl_strat=True, year_strat=True)

# #high_ldl
incidence_ldl_all_ages = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=False, year_strat=True)
incidence_ldl_all_ages_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=False, year_strat=True)
incidence_ldl_40to74 = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=False, year_strat=True)
incidence_ldl_40to74_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=False, year_strat=True)
incidence_ldl_75plus = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=False, year_strat=True)
incidence_ldl_75plus_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=False, year_strat=True)
incidence_ldl = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=False, year_strat=True)
incidence_ldl_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=False, year_strat=True)

# #total population
incidence_total_all_ages = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = False, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_total_all_ages_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = False, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_total_40to74 = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = False, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_total_40to74_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = False, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_total_75plus = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = False, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_total_75plus_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = False, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_total = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = False, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_total_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = False, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=True, year_strat=True)


In [ ]:
incidence_measures = pd.concat([incidence_htn_all_ages, incidence_htn_all_ages_both,
incidence_htn_40to74,
incidence_htn_40to74_both,
incidence_htn_75plus,
incidence_htn_75plus_both, incidence_htn,
incidence_htn_both,
incidence_db_all_ages,
incidence_db_all_ages_both,
incidence_db_40to74,
incidence_db_40to74_both,
incidence_db_75plus,
incidence_db_75plus_both,
incidence_db,
incidence_db_both,
incidence_ldl_all_ages,
incidence_ldl_all_ages_both,
incidence_ldl_40to74,
incidence_ldl_40to74_both,
incidence_ldl_75plus,
incidence_ldl_75plus_both,
incidence_ldl,
incidence_ldl_both,
incidence_total_all_ages,
incidence_total_all_ages_both,
incidence_total_40to74,
incidence_total_40to74_both,
incidence_total_75plus,
incidence_total_75plus_both, incidence_total, incidence_total_both])


In [ ]:
def get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True):

    measures = pd.concat([ylds, ylls, deaths, state])
    person_time = person_time.drop(columns=['measure'])
    measures['subpopulation'] = 'total_population'
    person_time['subpopulation'] = 'total_population'

    measures = measures[(measures.age_group != '30_to_34') & (measures.age_group != '35_to_39')]
    person_time = person_time[(person_time.age_group != '30_to_34') & (person_time.age_group != '35_to_39')]

    if causes_to_include != None:
        measures = measures[measures.cause.isin(causes_to_include)]
    if not risk_strat:
        measures = collapse_frame(measures, ['acs','fpg', 'sbp', 'ldl'])
        person_time = collapse_frame(person_time, ['acs','fpg', 'sbp', 'ldl'])
    if not age_strat_all_ages:
        measures = collapse_frame(measures, ['age_group'])
        measures['age_group'] = '40_plus'
        person_time = collapse_frame(person_time, ['age_group'])
        person_time['age_group'] = '40_plus'
    if not age_strat_75plus:
        measures_75 = measures[measures.age_group.isin(['75_to_79', '80_to_84', '85_to_89', '90_to_95', '95_plus'])]
        measures = collapse_frame(measures_75, ['age_group'])
        measures['age_group'] = '75_plus'
        pt_75 = person_time[person_time.age_group.isin(['75_to_79', '80_to_84', '85_to_89', '90_to_95', '95_plus'])]
        person_time = collapse_frame(pt_75, ['age_group'])
        person_time['age_group'] = '75_plus'
    if not age_strat_40_74:
        measures_40_74 = measures[measures.age_group.isin(['40_to_44', '45_to_49', '50_to_54', '55_to_59', '60_to_64', '65_to_69', '70_to_74'])]
        measures = collapse_frame(measures_40_74, ['age_group'])
        measures['age_group'] = '40_to_74'
        pt_40_74 = person_time[person_time.age_group.isin(['40_to_44', '45_to_49', '50_to_54', '55_to_59', '60_to_64', '65_to_69', '70_to_74'])]
        person_time = collapse_frame(pt_40_74, ['age_group'])
        person_time['age_group'] = '40_to_74'
    if not sex_strat:
        measures = collapse_frame(measures, ['sex'])
        measures['sex'] = 'both_sexes_combined'
        person_time = collapse_frame(person_time, ['sex'])
        person_time['sex'] = 'both_sexes_combined'
    if not cvd_strat:
        measures = collapse_frame(measures, ['cause'])
        measures['cause'] = 'CVD'
    if not loc_strat:
        measures = collapse_frame(measures, ['location'])
        measures['location'] = 'All locations'
        person_time = collapse_frame(person_time, ['location'])
        person_time['location'] = 'All locations'
    if not diabetes_strat:
        measures_dm = measures[(measures.fpg=='high')]
        measures = collapse_frame(measures_dm, ['acs','fpg', 'sbp', 'ldl'])
        measures['subpopulation'] = 'diabetic'
        person_time_dm = person_time[(person_time.fpg=='high')]
        person_time = collapse_frame(person_time_dm, ['acs','fpg', 'sbp', 'ldl'])
        person_time['subpopulation'] = 'diabetic'
    if not htn_strat:
        measures_htn = measures[(measures.sbp=='high')]
        measures = collapse_frame(measures_htn, ['acs','fpg', 'sbp', 'ldl'])
        measures['subpopulation'] = 'hypertensive'
        person_time_htn = person_time[(person_time.sbp=='high')]
        person_time = collapse_frame(person_time_htn, ['acs','fpg', 'sbp', 'ldl'])
        person_time['subpopulation'] = 'hypertensive' 
    if not high_ldl_strat:
        measures_ldl = measures[(measures.ldl=='high')]
        measures = collapse_frame(measures_ldl, ['acs','fpg', 'sbp', 'ldl'])
        measures['subpopulation'] = 'high_ldl'
        person_time_ldl = person_time[(person_time.ldl=='high')]
        person_time = collapse_frame(person_time_ldl, ['acs','fpg', 'sbp', 'ldl'])
        person_time['subpopulation'] = 'high_ldl'
    if not year_strat:
        measures = collapse_frame(measures, ['year'])
        measures['year'] = 'All years'
        person_time = collapse_frame(person_time, ['year'])
        person_time['year'] = 'All years'
    if not acmr_strat:
        measures_acmr = measures[(measures.measure == 'deaths')]
        measures = collapse_frame(measures_acmr, ['cause'])
        measures['cause'] = 'all_cause_mortality'
    if not daly_strat:
        measures_dalys = measures[((measures.measure == 'ylls') | (measures.measure == 'ylds'))]
        measures = collapse_frame(measures_dalys, ['measure'])
        measures['measure'] = 'dalys'

    measures = measures.set_index([x for x in person_time.columns if x != 'value'] + ['cause', 'measure']).value

    person_time = person_time.set_index([x for x in person_time.columns if x != 'value']).value
    rates = (measures / person_time * 100_000).reset_index()

    return rates

In [ ]:
# #htn strat
measures_htn_allages = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_htn_allages_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_htn_allages_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_htn_allages_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)
measures_htn_allages_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_htn_allages_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)

measures_htn_75plus = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_htn_75plus_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_htn_75plus_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_htn_75plus_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)
measures_htn_75plus_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_htn_75plus_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)

measures_htn_40to74 = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_htn_40to74_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_htn_40to74_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_htn_40to74_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)
measures_htn_40to74_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_htn_40to74_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)

measures_htn = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_htn_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_htn_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_htn_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)
measures_htn_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_htn_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)

# #diabetes strat
measures_db_allages = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_db_allages_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_db_allages_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_db_allages_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)
measures_db_allages_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_db_allages_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)

measures_db_75plus = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_db_75plus_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_db_75plus_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_db_75plus_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)
measures_db_75plus_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_db_75plus_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)

measures_db_40to74 = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_db_40to74_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_db_40to74_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_db_40to74_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)
measures_db_40to74_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_db_40to74_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)

measures_db = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_db_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_db_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_db_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)
measures_db_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_db_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)

# #High-LDL strat
measures_ldl_allages = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=True)
measures_ldl_allages_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=True)
measures_ldl_allages_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=False, daly_strat=True)
measures_ldl_allages_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=False)
measures_ldl_allages_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=False, daly_strat=True)
measures_ldl_allages_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=False)

measures_ldl_75plus = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=True)
measures_ldl_75plus_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=True)
measures_ldl_75plus_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=False, daly_strat=True)
measures_ldl_75plus_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=False)
measures_ldl_75plus_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=False, daly_strat=True)
measures_ldl_75plus_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=False)

measures_ldl_40to74 = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=True)
measures_ldl_40to74_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=True)
measures_ldl_40to74_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=False, daly_strat=True)
measures_ldl_40to74_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=False)
measures_ldl_40to74_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=False, daly_strat=True)
measures_ldl_40to74_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=False)

measures_ldl = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=True)
measures_ldl_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=True)
measures_ldl_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=False, daly_strat=True)
measures_ldl_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=False)
measures_ldl_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=False, daly_strat=True)
measures_ldl_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=False)

# #no risk strat
measures_total_allages = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_total_allages_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_total_allages_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_total_allages_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)
measures_total_allages_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_total_allages_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)

measures_total_75plus = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_total_75plus_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_total_75plus_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_total_75plus_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)
measures_total_75plus_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_total_75plus_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)

measures_total_40to74 = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_total_40to74_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_total_40to74_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_total_40to74_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)
measures_total_40to74_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_total_40to74_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)

measures_total = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_total_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_total_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_total_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)
measures_total_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_total_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)


In [ ]:
all_measures = pd.concat([measures_htn_allages, measures_htn_allages_both, measures_htn_allages_both_acmr, measures_htn_allages_both_dalys, measures_htn_allages_acmr, measures_htn_allages_dalys, measures_htn_75plus, measures_htn_75plus_both, measures_htn_75plus_both_acmr, measures_htn_75plus_both_dalys, measures_htn_75plus_acmr, measures_htn_75plus_dalys, measures_htn_40to74, measures_htn_40to74_both, measures_htn_40to74_both_acmr, measures_htn_40to74_both_dalys, measures_htn_40to74_acmr, measures_htn_40to74_dalys, measures_htn, measures_htn_both, measures_htn_both_acmr, measures_htn_both_dalys, measures_htn_acmr, measures_htn_dalys, measures_db_allages, measures_db_allages_both, measures_db_allages_both_acmr, measures_db_allages_both_dalys, measures_db_allages_acmr, measures_db_allages_dalys, measures_db_75plus, measures_db_75plus_both, measures_db_75plus_both_acmr, measures_db_75plus_both_dalys, measures_db_75plus_acmr, measures_db_75plus_dalys, measures_db_40to74, measures_db_40to74_both, measures_db_40to74_both_acmr, measures_db_40to74_both_dalys, measures_db_40to74_acmr, measures_db_40to74_dalys, measures_db, measures_db_both, measures_db_both_acmr, measures_db_both_dalys, measures_db_acmr, measures_db_dalys, measures_ldl_allages, measures_ldl_allages_both, measures_ldl_allages_both_acmr, measures_ldl_allages_both_dalys, measures_ldl_allages_acmr, measures_ldl_allages_dalys, measures_ldl_75plus, measures_ldl_75plus_both, measures_ldl_75plus_both_acmr, measures_ldl_75plus_both_dalys, measures_ldl_75plus_acmr, measures_ldl_75plus_dalys, measures_ldl_40to74, measures_ldl_40to74_both, measures_ldl_40to74_both_acmr, measures_ldl_40to74_both_dalys, measures_ldl_40to74_acmr, measures_ldl_40to74_dalys, measures_ldl, measures_ldl_both, measures_ldl_both_acmr, measures_ldl_both_dalys, measures_ldl_acmr, measures_ldl_dalys, measures_total_allages, measures_total_allages_both, measures_total_allages_both_acmr, measures_total_allages_both_dalys, measures_total_allages_acmr, measures_total_allages_dalys, measures_total_75plus, measures_total_75plus_both, measures_total_75plus_both_acmr, measures_total_75plus_both_dalys, measures_total_75plus_acmr, measures_total_75plus_dalys, measures_total_40to74, measures_total_40to74_both, measures_total_40to74_both_acmr, measures_total_40to74_both_dalys, measures_total_40to74_acmr, measures_total_40to74_dalys, measures_total , measures_total_both, measures_total_both_acmr, measures_total_both_dalys, measures_total_acmr , measures_total_dalys])


In [ ]:
def get_diff_from_baseline(all_measures, incidence_measures):
    df = all_measures
    df = df.append(incidence_measures, ignore_index=True)
    baseline = df[df.scenario == 'baseline']
    baseline = baseline.rename(columns = {'value':'baseline'})
    df = pd.merge(df, baseline, on=['age_group', 'sex', 'year', 'cause', 'input_draw', 'location', 'measure', 'subpopulation'], how='left')
    df['difference_from_baseline_count'] = df['baseline'] - df['value']    
    df = df.rename(columns={'scenario_x':'scenario'})
    df = df.drop(columns=['scenario_y', 'baseline'])
    return df

In [ ]:
df = get_diff_from_baseline(all_measures, incidence_measures)

In [ ]:
def quantile_(q):
    """Factory function for pandas gb aggregate.
    """
    def q_factory(df):
        return df.quantile(q)
    q_factory.__name__ = f"Q{q:0.3f}"
    return q_factory

def report_aggregates(df):
    gb = df.groupby(["age_group", "sex", "year", "cause", "location", "measure", "scenario", "subpopulation"], as_index=False)
    metrics = ["mean", quantile_(0.025), quantile_(0.975)]
    df = gb.agg({"value": metrics, "difference_from_baseline_count": metrics})
    df.columns = list(map(lambda c: "_".join(filter(None, c)), df.columns))
    return df

In [ ]:
aggregates = report_aggregates(df)
aggregates_formatting = aggregates

In [ ]:
def get_table_formatting(aggregates_formatting):    
    df = aggregates
    df['subpopulation'] = df.subpopulation.str.replace('high_ldl','LDL-c > 4.9')
    df['subpopulation'] = df.subpopulation.str.replace('hypertensive','Hypertension (SBP > 140 mmHg)')
    df['subpopulation'] = df.subpopulation.str.replace('diabetic','Diabetes (FPG > 7.0 mmol/l)')
    df['subpopulation'] = df.subpopulation.str.replace('total_population','Entire population')
    df['sex'] = df.sex.map(mapping_dict_data['sex'])
    df = df[(df.cause != 'chronic_kidney_disease') & (df.cause != 'diabetes_mellitus') & (df.cause != 'other_causes') & (df.measure != 'prevalence')]
    ihd = df[(df.cause == 'ischemic_heart_disease')]
    ihd['measure'] = ihd.measure.str.replace('dalys', 'DALYs due to IHD')
    ihd['measure'] = ihd.measure.str.replace('ylls', 'YLLs due to IHD')
    ihd['measure'] = ihd.measure.str.replace('ylds', 'YLDs due to IHD')
    ihd['measure'] = ihd.measure.str.replace('deaths', 'Mortality due to IHD')
    ihd['measure'] = ihd.measure.str.replace('incidence', 'Incidence of IHD')

    i_stroke = df[(df.cause == 'ischemic_stroke')]
    i_stroke['measure'] = i_stroke.measure.str.replace('dalys', 'DALYs due to Ischemic stroke')
    i_stroke['measure'] = i_stroke.measure.str.replace('ylls', 'YLLs due to Ischemic stroke')
    i_stroke['measure'] = i_stroke.measure.str.replace('ylds', 'YLDs due to Ischemic stroke')
    i_stroke['measure'] = i_stroke.measure.str.replace('deaths', 'Mortality due to Ischemic stroke')
    i_stroke['measure'] = i_stroke.measure.str.replace('incidence', 'Incidence of Ischemic stroke')
    acmr = df[(df.cause == 'all_cause_mortality')]
    acmr['measure'] = acmr.measure.str.replace('deaths', 'All-cause mortality')
    df = pd.concat([ihd, i_stroke, acmr])
    df = df.drop(columns=['cause'])
    
    df['location'] = df.location.map(mapping_dict_data['location'])
    df['sex'] = df.sex.str.capitalize()
    df = df.rename(columns={'measure':'Outcome'})
    df = df[(df.age_group != '75_to_79') & (df.age_group != '80_to_84') & (df.age_group != '85_to_89') & (df.age_group != '90_to_94') & (df.age_group != '95_plus')]
    df['age_group'] = df.age_group.str.replace('_', ' ')
    df['age_group'] = df.age_group.str.replace(' plus', '+')
    df['scenario'] = df.scenario.map(mapping_dict_data['scenario'])
    df = df.rename(columns={'age_group':'Age group', 'cause':'Cause', 'sex':'Sex', 'scenario':'Scenario', 'location':'Location name', 
                            'subpopulation':'Subpopulation (measured at start of simulation)', 'measure':'Measure', 'year':'Year', 
                            'value_mean':'Mean estimated rate (per 100,000 person-years)', 
                            'value_Q0.025':'Lower uncertainty interval (per 100,000 person-years)', 
                            'value_Q0.975':'Upper uncertainty interval (per 100,000 person-years)', 
                            'difference_from_baseline_count_mean':'Difference from baseline- mean rate (per 100,000 person-years)',
                            'difference_from_baseline_count_Q0.025':'Difference from baseline- upper uncertainty interval (per 100,000 person-years)', 
                            'difference_from_baseline_count_Q0.975':'Difference from baseline- lower uncertainty interval (per 100,000 person-years)'})
    df = df[['Location name', 'Year', 'Subpopulation (measured at start of simulation)', 'Age group', 'Sex', 'Scenario', 'Outcome',  'Mean estimated rate (per 100,000 person-years)', 'Lower uncertainty interval (per 100,000 person-years)', 'Upper uncertainty interval (per 100,000 person-years)',  'Difference from baseline- mean rate (per 100,000 person-years)', 'Difference from baseline- lower uncertainty interval (per 100,000 person-years)', 'Difference from baseline- upper uncertainty interval (per 100,000 person-years)']]
    df['Sex'] = df.Sex.str.replace('Both sexes','Both')
    return df

disease_outcomes = get_table_formatting(aggregates_formatting)
disease_outcomes.to_csv(str(save_path_j)+'/full_output_table_disease_outcomes.csv', index=False)


# Treatment type and status table

In [ ]:
med_options = ['fibrates',
       'ezetimibe', 'none','low_potency_statin_low_dose',
       'low_potency_statin_high_dose',
       'high_potency_statin_low_dose',
       'high_potency_statin_high_dose',
       'low_potency_statin_low_dose_multi',
       'low_potency_statin_high_dose_multi',
       'high_potency_statin_low_dose_multi',
       'high_potency_statin_high_dose_multi',
       'low_potency_statin_low_dose_fdc',
       'low_potency_statin_high_dose_fdc',
       'high_potency_statin_low_dose_fdc',
       'high_potency_statin_high_dose_fdc']

In [ ]:
def get_med_pcts(person_time_misc, med_options):
    meds = pd.DataFrame([])
    for med in med_options:
        df = person_time_misc[(person_time_misc.measure == f'{med}_person_time')]
        df['medication'] = f'{med}'
        meds = meds.append(df)
    df = meds.groupby(['age_group', 'sex', 'year', 'input_draw', 'location', 'scenario', 'medication']).value.sum().reset_index()
    df_all = df.groupby(['age_group', 'sex', 'year', 'input_draw', 'location', 'scenario']).value.sum().reset_index()
    index_columns = ['age_group', 'sex', 'year', 'input_draw','location', 'scenario']
    df = df.set_index(index_columns + ['medication']).value
    df_all = df_all.set_index(index_columns).value
    med_prop = df / df_all
    med_prop = med_prop.reset_index()
    med_prop['location'] = med_prop.location.map(mapping_dict_data['location'])
    med_prop['scenario'] = med_prop.scenario.map(mapping_dict_data['scenario'])
    med_prop = med_prop.rename(columns={'scenario':'Scenario'})

    return med_prop


In [ ]:
meds = get_med_pcts(person_time_misc, med_options)

In [ ]:
def get_med_pcts_primary_prevention(person_time_misc, med_options, acs_strat=False):
    meds = pd.DataFrame([])
    for med in med_options:
        df = person_time_misc[(person_time_misc.measure == f'{med}_person_time')]
        df['medication'] = f'{med}'
        meds = meds.append(df)
    df = meds
    df_all = df.groupby(['age_group', 'sex', 'year', 'input_draw', 'location', 'scenario', 'acs', 'fpg', 'sbp', 'ldl']).value.sum().reset_index()
    
    if not acs_strat:
        df = df[(df.acs != 'post') & (df.medication != 'none')]
        df = collapse_frame(df, ['medication','acs','fpg', 'sbp', 'ldl'])
        df['Treatment type'] = '% of simulants on Primary Prevention Tx'
        df_all = collapse_frame(df_all, ['acs','fpg', 'sbp', 'ldl'])
        df_all['Treatment type'] = '% of simulants on Primary Prevention Tx'
        
    index_columns = ['age_group', 'sex', 'year', 'input_draw','location', 'scenario', 'acs', 'fpg', 'sbp', 'ldl']
    df = df.set_index([x for x in df_all.columns if x != 'value']).value
    df_all = df_all.set_index([x for x in df_all.columns if x != 'value']).value
    med_prop = (df / df_all).reset_index()
    med_prop['location'] = med_prop.location.map(mapping_dict_data['location'])
    med_prop['scenario'] = med_prop.scenario.map(mapping_dict_data['scenario'])
    med_prop = med_prop.rename(columns={'scenario':'Scenario'})
    med_prop = med_prop.groupby(['location', 'Scenario', 'Treatment type', 'year']).value.mean().reset_index()
    return med_prop


In [ ]:
meds_primary_prevention = get_med_pcts_primary_prevention(person_time_misc, med_options, acs_strat=False)
meds_primary_prevention_2020 = meds_primary_prevention[(meds_primary_prevention.year =='2020')]
meds_primary_prevention_2024 = meds_primary_prevention[(meds_primary_prevention.year =='2024')]


In [ ]:
def get_table_6(locations, meds, med_options, meds_primary_prevention_2020): 
    init_df = pd.DataFrame([])
    shared_cols = ['location', 'current_prescription']
    med_df = meds[(meds.year == '2020')]
    med_df = med_df.groupby(['location', 'Scenario', 'medication']).value.mean().reset_index()
    
    agg_fibrates_ezetimibe = med_df[(med_df.medication == 'fibrates') | (med_df.medication == 'ezetimibe')]
    agg_fibrates_ezetimibe = agg_fibrates_ezetimibe.groupby(['location', 'Scenario']).value.sum().reset_index()
    agg_fibrates_ezetimibe['Treatment type'] = '% other (fibrates, ezetimibe alone)'
    
    agg_hp = med_df[(med_df.medication.str.contains('high_potency_statin'))]
    agg_hp = agg_hp.groupby(['location', 'Scenario']).value.sum().reset_index()
    agg_hp['Treatment type'] = '% HP'
    
    agg_lp = med_df[(med_df.medication.str.contains('low_potency_statin'))]
    agg_lp = agg_lp.groupby(['location', 'Scenario']).value.sum().reset_index()
    agg_lp['Treatment type'] = '% LP'
    
    agg_fdc = med_df[(med_df.medication.str.contains('fdc'))]
    agg_fdc = agg_fdc.groupby(['location', 'Scenario']).value.sum().reset_index()
    agg_fdc['Treatment type'] = '% of simulants on FDC'
    
    agg_any = med_df[(med_df.medication != 'none')]
    agg_any = agg_any.groupby(['location', 'Scenario']).value.sum().reset_index()
    agg_any['Treatment type'] = '% of simulants on any lipid lowering treatment'
    
    agg_multi = med_df[(med_df.medication.str.contains('fdc')) | (med_df.medication.str.contains('multi'))]
    agg_multi = agg_multi.groupby(['location', 'Scenario']).value.sum().reset_index()
    agg_multi['Treatment type'] = '% on multiple DRUGS (note: NOT multi-pills)'
    
    agg_none = med_df[(med_df.medication == 'none')]
    agg_none = agg_none.groupby(['location', 'Scenario']).value.sum().reset_index()
    agg_none['Treatment type'] = '% of simulants not on any lipid lowering treatment'
    
    treatment_table = pd.concat([agg_fibrates_ezetimibe, agg_hp, agg_lp, agg_fdc, agg_any, agg_multi, agg_none])
    treatment_table['year'] = '2020'
    treatment_table = treatment_table.append(meds_primary_prevention_2020, ignore_index=True)
    treatment_table = treatment_table.rename(columns={'value':'Start of simulation, Jan 1., 2020 (same for all scenarios)'})

    treatment_table = treatment_table.round(2)
    return treatment_table


In [ ]:
def get_table_6_end(locations, meds, med_options, meds_primary_prevention_2024): 
    init_df = pd.DataFrame([])
    shared_cols = ['location', 'current_prescription']
    med_df = meds[(meds.year == '2024')]
    med_df = med_df.groupby(['location', 'Scenario', 'medication']).value.mean().reset_index()
    
    agg_fibrates_ezetimibe = med_df[(med_df.medication == 'fibrates') | (med_df.medication == 'ezetimibe')]
    agg_fibrates_ezetimibe = agg_fibrates_ezetimibe.groupby(['location', 'Scenario']).value.sum().reset_index()
    agg_fibrates_ezetimibe['Treatment type'] = '% other (fibrates, ezetimibe alone)'
    
    agg_hp = med_df[(med_df.medication.str.contains('high_potency_statin'))]
    agg_hp = agg_hp.groupby(['location', 'Scenario']).value.sum().reset_index()
    agg_hp['Treatment type'] = '% HP'
    
    agg_lp = med_df[(med_df.medication.str.contains('low_potency_statin'))]
    agg_lp = agg_lp.groupby(['location', 'Scenario']).value.sum().reset_index()
    agg_lp['Treatment type'] = '% LP'
    
    agg_fdc = med_df[(med_df.medication.str.contains('fdc'))]
    agg_fdc = agg_fdc.groupby(['location', 'Scenario']).value.sum().reset_index()
    agg_fdc['Treatment type'] = '% of simulants on FDC'
    
    agg_any = med_df[(med_df.medication != 'none')]
    agg_any = agg_any.groupby(['location', 'Scenario']).value.sum().reset_index()
    agg_any['Treatment type'] = '% of simulants on any lipid lowering treatment'
    
    agg_multi = med_df[(med_df.medication.str.contains('fdc')) | (med_df.medication.str.contains('multi'))]
    agg_multi = agg_multi.groupby(['location', 'Scenario']).value.sum().reset_index()
    agg_multi['Treatment type'] = '% on multiple DRUGS (note: NOT multi-pills)'
    
    agg_none = med_df[(med_df.medication == 'none')]
    agg_none = agg_none.groupby(['location', 'Scenario']).value.sum().reset_index()
    agg_none['Treatment type'] = '% of simulants not on any lipid lowering treatment'
    
    treatment_table = pd.concat([agg_fibrates_ezetimibe, agg_hp, agg_lp, agg_fdc, agg_any, agg_multi, agg_none])
    treatment_table['year'] = '2024'
    treatment_table = treatment_table.append(meds_primary_prevention_2024, ignore_index=True)
    treatment_table = treatment_table.rename(columns={'value':'End of simulation, Dec. 31, 2024', 'location':'Location'})
    treatment_table = treatment_table[['Location', 'Scenario', 'Treatment type', 'End of simulation, Dec. 31, 2024']]

    treatment_table = treatment_table.round(2)
    return treatment_table


In [ ]:
table_6 = get_table_6(locations, meds, med_options, meds_primary_prevention_2020)

In [ ]:
table_6_all = table_6.groupby(['location', 'Treatment type'])['Start of simulation, Jan 1., 2020 (same for all scenarios)'].mean().reset_index()
table_6_all = table_6_all.round(2)
table_6_all = table_6_all.rename(columns={'location':'Location'})

In [ ]:
table_6_2024 = get_table_6_end(locations, meds, med_options, meds_primary_prevention_2024)
table_6_2024_bau = table_6_2024[(table_6_2024.Scenario == 'Business as Usual')]
table_6_2024_bau = table_6_2024_bau.rename(columns={'End of simulation, Dec. 31, 2024':'BAU'})
table_6_2024_bau = table_6_2024_bau.drop(columns=['Scenario'])
table_6_2024_int_1 = table_6_2024[(table_6_2024.Scenario == 'Intervention 1 (multiple pills)')]
table_6_2024_int_1 = table_6_2024_int_1.rename(columns={'End of simulation, Dec. 31, 2024':'Scenario 1 (Multiple Pills)'})
table_6_2024_int_1 = table_6_2024_int_1.drop(columns=['Scenario'])
table_6_2024_int_2 = table_6_2024[(table_6_2024.Scenario == 'Intervention 2 (FDC)')]
table_6_2024_int_2 = table_6_2024_int_2.rename(columns={'End of simulation, Dec. 31, 2024':'Scenario 2 (FDC)'})
table_6_2024_int_2 = table_6_2024_int_2.drop(columns=['Scenario'])
shared_cols = ['Location','Treatment type']
end_sim_scenario_df = pd.merge(table_6_2024_bau, table_6_2024_int_1, left_on=shared_cols, right_on=shared_cols, how='left')
end_sim_scenario_df = pd.merge(end_sim_scenario_df, table_6_2024_int_2, left_on=shared_cols, right_on=shared_cols, how='left')

full_treatment_table = pd.merge(table_6_all,end_sim_scenario_df, left_on=shared_cols, right_on=shared_cols, how='left')

In [ ]:
full_treatment_table.to_csv(str(save_path_j)+'/treatment_type_table.csv', index=False)